## Import libraries & load data:

In [1]:
import pandas as pd
import numpy as np

In [9]:
# Load datasets

customers = pd.read_csv('customers.csv')
accounts = pd.read_csv('accounts.csv')
transactions = pd.read_csv('transactions.csv')
loans = pd.read_csv('loans.csv')

In [10]:
# Data preparation

transactions['transaction_date'] = pd.to_datetime(
    transactions['transaction_date'], 
    errors='coerce',
    dayfirst=True
)

## Calculate Net Revenue per Account:

In [14]:
transactions['signed_amount'] = np.where(
    transactions['transaction_type'] == 'Credit',
    transactions['transaction_amount'],
    -transactions['transaction_amount']
)

account_revenue = (
    transactions
    .groupby('account_id', as_index=False)['signed_amount']
    .sum()
    .rename(columns={'signed_amount': 'net_revenue'})
)
account_revenue.head()

,account_id,net_revenue
0,A001,7000
1,A002,9000
2,A003,28000
3,A004,6000
4,A005,47000


## Customer-level Revenue:

In [16]:
customer_revenue = (
    accounts
    .merge(account_revenue, on='account_id', how='left')
    .groupby('customer_id', as_index=False)['net_revenue']
    .sum()
)
customer_revenue.head()

,customer_id,net_revenue
0,C001,7000.0
1,C002,9000.0
2,C003,28000.0
3,C004,6000.0
4,C005,47000.0


## customer_revenue:

In [17]:
customer_analysis = customers.merge(
    customer_revenue, on='customer_id', how='left'
)

customer_analysis['net_revenue'] = customer_analysis['net_revenue'].fillna(0)

## Add Risk Flag (from loans):

In [18]:
loan_risk = (
    loans.groupby('customer_id', as_index=False)['default_flag']
    .max()
)

customer_analysis = customer_analysis.merge(
    loan_risk, on='customer_id', how='left'
)

customer_analysis['default_flag'] = customer_analysis['default_flag'].fillna(0)

## Customer Segmentation:

In [19]:
def segment_customer(row):
    if row['net_revenue'] > 50000 and row['credit_score'] >= 700:
        return 'High Value – Low Risk'
    elif row['net_revenue'] > 50000 and row['credit_score'] < 700:
        return 'High Value – High Risk'
    elif row['net_revenue'] <= 50000 and row['credit_score'] >= 700:
        return 'Low Value – Low Risk'
    else:
        return 'Low Value – High Risk'

customer_analysis['customer_segment'] = customer_analysis.apply(
    segment_customer, axis=1
)

In [27]:
# Save processed dataset for Power BI

customer_analysis.to_csv(
    'C:/Users/HP/Downloads/customer_analysis_final.csv',
    index=False
)


# # Export final customer-level dataset
# for dashboarding and business insights